In [1]:
!pip install pyngrok
!pip install flask
!pip install flask-ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=c92e5e38874f70787b7a77a4bb2d39233ae70bde658e0ba054ed52f98e77c843
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [2]:
! pip install Flask-Cors
! pip install python-dotenv

In [3]:
! git clone https://github.com/binary-hood/ChatBot.git

Cloning into 'ChatBot'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 0), reused 12 (delta 0), pack-reused 0
Receiving objects: 100% (12/12), 7.91 KiB | 1.58 MiB/s, done.


In [4]:
import os

In [5]:
%cd ChatBot

/content/ChatBot


In [6]:
! pip install flask-socketio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 814.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 5.3 MB/s eta 0:00:00


In [7]:
!pip install simple-websocket

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [8]:
! pip install python-socketio

In [9]:
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.9 MB/s eta 0:00:00


#**Module setup and Modle dumping**

In [10]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from typing import Any
from functools import wraps
import os
import datetime
from dotenv import load_dotenv
import threading
from flask import Flask, render_template, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import jsonschema
from jsonschema import validate
from flask_socketio import join_room, leave_room, send, SocketIO, emit
from random import random
from time import sleep
from threading import Thread, Event

In [11]:
import pickle


#**Load model DialoGPT-large from Hugging face**

In [12]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

#**Dump model in pickle file**

In [13]:
pickle.dump(model, open('model_GPT2.pkl', 'wb'))
pickle.dump(tokenizer, open('tokenizer_GPT2.pkl', 'wb'))

#**Load Modle and Data Response**

In [14]:
model_m = pickle.load(open('/content/ChatBot/model_GPT2.pkl','rb'))
model_tokenizer = pickle.load(open('/content/ChatBot/tokenizer_GPT2.pkl','rb'))

In [15]:
# def load_model_GPT2():
#     """
#     method will load the model and return loaded model model_GPT2.

#     Parameters:
#     ----------
#     None

#     Return:
#     ------
#     model: Any
#         return the model object loaded from model_GPT2.pkl file.

#     """
#     model = pickle.load(open('/content/ChatBot/model_GPT2.pkl','rb'))
#     return model

# def load_GPT2_tokenizer():
#     """
#     method will load the model and return loaded model tokenizer_GPT2.

#     Parameters:
#     ----------
#     None

#     Return:
#     ------
#     model: Any
#         return the model object loaded from tokenizer_GPT2.pkl file.

#     """
#     model_tokenizer = pickle.load(open('/content/ChatBot/tokenizer_GPT2.pkl','rb'))
#     return model_tokenizer

def get_Chat_response(text_data: str)-> dict:

    """
    method will take the text_data and return its converstaional response by using the
    GPT2 model.

    Parameters:
    ----------
    text_data: str
      text data contain the message send by the user to the chatbot.

    Return:
    ------
    str
        return the last response send by the chatbot as a string.

    """
    model = model_m
    tokenizer = model_tokenizer
    for step in range(6):
        # encode the new user input, add the eos_token and return a tensor in Pytorch
        new_user_input_ids = tokenizer.encode(str(text_data) + tokenizer.eos_token, return_tensors = 'pt')
        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim = -1) if step > 0 else new_user_input_ids
        # generated a response while limiting the total chat history to 1000 tokens,
        chat_history_ids = model.generate(bot_input_ids, max_length = 1000, pad_token_id = tokenizer.eos_token_id)
        # pretty print last ouput tokens from bot
        return tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens = True)

#**Main App**

In [16]:
"""
Main module contain the major fuctionalities of the application.
"""

ngrok.set_auth_token("2T1O4uqRG1iUOaqPtGGsOIaTysW_5vcKQETZ7wVzDYebPxHn")
app = Flask(__name__)
run_with_ngrok(app)

socketio = SocketIO(app)
thread = Thread()
thread_stop_event = Event()

@app.route("/")
def index():
    """
    default route.

    Parameters:
    ----------
    None

    Return:
    ------
    template
        render tempelate chat.html.

    """
    return render_template('chat.html')


@app.route("/get", methods=["GET", "POST"])
def chat():
    """
    method will take the data and return the response in text form.

    Parameters:
    ----------
    None

    Return:
    ------
    str
        return the response in return of user text data.

    """
    message_data = request.form["msg"]
    return get_Chat_response(message_data)

@socketio.on('message')
def handle_message(rawText: str):
    print(get_Chat_response(rawText))

if __name__ == "__main__":
    # socketio.run(app,allow_unsafe_werkzeug = True)
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5147-34-73-78-238.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [15/Aug/2023 06:01:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Aug/2023 06:01:51] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Aug/2023 06:01:55] "GET /favicon.ico HTTP/1.1" 404 -
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
INFO:werkzeug:127.0.0.1 - - [15/Aug/2023 06:02:09] "POST /get HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Aug/2023 06:02:10] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Aug/2023 06:02:11] "GET /static/style.css HTTP/1.1" 304 -
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
INFO:werkzeug:127.0.0.1 - - [15/Aug/2023 06:02:18] "POST /get HTTP/1.1" 200 -
A decoder-only architecture is being used, but right-padding was detected! For correc